In [159]:
import librosa
import simfile
import numpy as np
import pandas as pd

In [100]:
sound_file, sr = librosa.load("../pack_37_9ddada/Butterfly/Butterfly.ogg")

In [58]:
input_filename = "../pack_37_9ddada/Butterfly/Butterfly.sm"

In [60]:
def _find_chart(charts, stepstype, difficulty):
    for chart in charts:
        if chart.stepstype == stepstype and chart.difficulty == difficulty:
            return chart
    return False

In [61]:
def filter_charts(input_filename, stepstype, difficulty):
    smfile = simfile.open(input_filename)
    target = _find_chart(smfile.charts, stepstype, difficulty)
    if target:
        smfile.charts = [target]
        return smfile
    else:
        print(f"No simfile of type {stepstype} and difficulty {difficulty} found.")
        return None


In [62]:
filter_charts(butterfly, "dance-single", "Hard")

In [65]:
with simfile.mutate(
    input_filename,
    backup_filename=f'{input_filename}.old',
) as f:
    if f.subtitle.endswith('(edited)'):
        raise simfile.CancelMutation
    filter_charts(f, "dance-single", "Hard")
    f.subtitle += '(edited)'

In [120]:
from simfile.timing import Beat, TimingData
from simfile.notes import NoteData
from simfile.notes.timed import time_notes
f = simfile.open(input_filename)
note_data = NoteData(f.charts[0])
timing_data = TimingData(f, f.charts[0])
note_time_pairs = []
for timed_note in time_notes(note_data, timing_data):
    note_time_pairs.append((timed_note.note.column, timed_note.time))
note_time_pairs = np.array(note_time_pairs)
print(note_time_pairs)


[[ 1.         14.35052075]
 [ 2.         14.57287144]
 [ 2.         14.79522213]
 [ 1.         15.23992351]
 [ 0.         15.68462489]
 [ 3.         15.68462489]
 [ 0.         16.12932628]
 [ 1.         16.35167697]
 [ 2.         16.57402766]
 [ 1.         17.01872904]
 [ 0.         17.46343043]
 [ 3.         17.46343043]
 [ 3.         17.90813181]
 [ 1.         18.1304825 ]
 [ 0.         18.35283319]
 [ 1.         18.79753458]
 [ 3.         19.24223596]
 [ 1.         19.68693734]
 [ 0.         19.90928803]
 [ 1.         20.13163872]
 [ 3.         20.57634011]
 [ 0.         21.02104149]
 [ 1.         21.46574287]
 [ 3.         21.68809357]
 [ 2.         21.91044426]
 [ 1.         22.35514564]
 [ 0.         22.79984702]
 [ 3.         22.79984702]
 [ 3.         23.24454841]
 [ 1.         23.4668991 ]
 [ 2.         23.68924979]
 [ 1.         24.13395117]
 [ 0.         24.57865255]
 [ 3.         24.57865255]
 [ 0.         25.02335394]
 [ 1.         25.24570463]
 [ 3.         25.46805532]
 

In [104]:
duration = librosa.get_duration(sound_file)

duration_minutes = duration / 60.0

print(duration_minutes)

1.5664761904761906


/var/folders/87/fylw6rqx7cz0vrk0z9cdsbhh0000gn/T/ipykernel_40749/3376991511.py:1: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.5100899e-05 9.9585202e-05
 1.0719002e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  duration = librosa.get_duration(sound_file)


In [148]:
chroma = librosa.feature.chroma_stft(y=sound_file, sr=sr)
verify = (duration * sr) / len(chroma[0])
target_map = [4] * len(chroma[0])
secs_per_sample = 1.0 / sr
total_samples = int(duration * sr)
print(secs_per_sample)

for i in range(len(chroma[0])):
    sample_num = 512 * i
    time_in_song = sample_num * secs_per_sample
    for idx, item in enumerate(note_time_pairs):
        if (item[1] < (time_in_song + (secs_per_sample * 256))) and (item[1] > (time_in_song - (secs_per_sample * 256))):
            target_map[i] = item[0]
print(512 * len(chroma[0]))
print((sr * duration) * secs_per_sample)

4.5351473922902495e-05
2072576
93.98857142857143


In [157]:
print(chroma.shape)
chroma = chroma.T
print(chroma.shape)

(12, 4048)
(4048, 12)


In [158]:
target_map = np.array(target_map)

In [162]:
butterfly_df = pd.DataFrame(chroma)

In [163]:
butterfly_df["target"] = target_map

In [168]:
butterfly_df.to_csv("../data/butterfly.csv")

In [169]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(chroma, target_map)

LinearRegression()

In [183]:
from sklearn.neighbors import KNeighborsClassifier

neighbors = KNeighborsClassifier(n_neighbors=3)
neighbors.fit(chroma, target_map)

KNeighborsClassifier(n_neighbors=3)

In [172]:
my_fire, sr = librosa.load("../pack_37_9ddada/My Fire (UKS Remix)/My Fire (UKS Remix).ogg")

In [174]:
fire_chroma = librosa.feature.chroma_stft(y=my_fire, sr=sr)

In [175]:
fire_chroma = fire_chroma.T

In [180]:
fire_prediction = regressor.predict(fire_chroma)
fire_prediction = np.around(fire_prediction)

In [181]:
fire_pred_df = pd.DataFrame(fire_prediction)


In [182]:
fire_pred_df.to_csv("../data/fire_prediction.csv")

In [187]:
neighbor_result = []
for item in fire_chroma:
    neighbor_result.append(neighbors.predict(item.reshape(1,-1)))

In [189]:
for i in range(len(neighbor_result)):
    neighbor_result[i] = neighbor_result[i][0]

In [190]:
print(neighbor_result)

[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 2.0, 1.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0,